In [2]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [3]:
# dij需求i到设施j的成本
# d = [[1, 2], [1, 2]]
# n = 2   # 两个设施点
# m = 2   # 两个需求点
d = [[1, 2], [3, 4], [5, 6]]
n = 2   # 两个设施点
m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [4]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [5]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [6]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    # for i in range(m):
    #     Ht = np.zeros((2**num_qubits, 2**num_qubits))
    #     for j in range(n):
    #         Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
    #     Ht -= np.eye(2**num_qubits)
    #     Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [7]:
def gnrt_Hd(n):
  Hd = np.zeros((2**n, 2**n)).astype(np.complex128)
  for i in range(n):
    j = (i + 1) % n
    Hd += (add_in_target(n, i, GateX) @ add_in_target(n, j, GateX) + add_in_target(n, i, GateY) @ add_in_target(n, j, GateY))
  return -Hd


In [8]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in range(m * n):
    qc.h(i)
  for i in range(m * n, m * n * 2, 2):
    qc.x(i)
  for i in range(m * n * 2, num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits)) # transpile
    # 
    for i in range(m * n):
      qc.rx(beta[dp], i)
    for i in range(m):
      qc.unitary(expm(-1j * beta[dp] * gnrt_Hd(n)), range(n * (m + i) , n * (m + i + 1)))
    for i in range(2 * m * n, num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [9]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

         ┌───┐┌───────────┐┌─────────┐ ┌───────────┐┌─────────┐ ┌───────────┐»
    q_0: ┤ H ├┤0          ├┤ Rx(π/4) ├─┤0          ├┤ Rx(π/4) ├─┤0          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_1: ┤ H ├┤1          ├┤ Rx(π/4) ├─┤1          ├┤ Rx(π/4) ├─┤1          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_2: ┤ H ├┤2          ├┤ Rx(π/4) ├─┤2          ├┤ Rx(π/4) ├─┤2          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_3: ┤ H ├┤3          ├┤ Rx(π/4) ├─┤3          ├┤ Rx(π/4) ├─┤3          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_4: ┤ H ├┤4          ├┤ Rx(π/4) ├─┤4          ├┤ Rx(π/4) ├─┤4          ├»
         ├───┤│           │├─────────┤ │           │├─────────┤ │           │»
    q_5: ┤ H ├┤5          ├┤ Rx(π/4) ├─┤5          ├┤ Rx(π/4) ├─┤5          ├»
         ├───┤│           │├─────────┴┐│           │

In [10]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  # for i in range(m):
  #   t = 0
  #   for j in range(n):
  #     t += num[n * (1 + i) + j]
  #   C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [11]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [12]:
test(40, 1, np.full(1 * 2, np.pi))

Iteration 10, Result: 291.945
Iteration 20, Result: 145.1395
Iteration 30, Result: 95.676
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 95.676
       x: [ 4.358e+00  2.190e+00]
    nfev: 31
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
10101010000000 27.7% 11
00101010000000 6.4% 129
10101010000010 6.0% 51
10101010001000 5.9% 51
10101010100000 5.8% 51
11101010000000 5.5% 132
10101010001010 1.9% 91
10101010101000 1.8% 91
10101010100010 1.8% 91
10101010000001 1.7% 51
10101010010000 1.7% 51
10101010000100 1.7% 51
01101010000000 1.2% 250
11101010000010 1.2% 172
11101010100000 1.2% 172
11101010001000 1.2% 172
00101010000010 1.0% 249
00101010100000 1.0% 249
00101010001000 0.9% 249
00101010001010 0.8% 369


In [13]:
test(40, 2, np.full(2 * 2, np.pi))

Iteration 10, Result: 288.5185
Iteration 20, Result: 387.6655
Iteration 30, Result: 162.0285
Iteration 40, Result: 162.8235
Iteration 50, Result: 160.5625
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 160.5625
       x: [ 4.207e+00  3.146e+00  4.144e+00  3.172e+00]
    nfev: 51
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
00101010000000 4.9% 129
01101010010101 4.3% 130
01101010000000 2.8% 250
01101010010001 2.0% 170
01101010000101 1.9% 170
01101010010100 1.9% 170
01101010010000 1.7% 210
01101010000001 1.7% 210
01101010000100 1.7% 210
00101010010101 1.6% 249
00100110000000 1.3% 130
00011010000000 1.3% 130
00101001000000 1.3% 130
10101010000010 1.1% 51
10101010001000 1.1% 51
10101010100000 1.1% 51
01101001010101 1.0% 131
01011010010101 1.0% 131
01100110010101 0.9% 131
11101010010111 0.9% 52


In [14]:
test(40, 3, np.full(3 * 2, np.pi))

Iteration 10, Result: 229.558
Iteration 20, Result: 259.9105
Iteration 30, Result: 206.9015
Iteration 40, Result: 211.186
Iteration 50, Result: 132.9385
Iteration 60, Result: 117.497
Iteration 70, Result: 114.5275
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 115.1105
       x: [ 4.155e+00  3.162e+00  4.145e+00  3.130e+00  4.174e+00
            3.384e+00]
    nfev: 79
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
10101010000000 17.4% 11
10011010000000 6.8% 92
10101001000000 6.7% 92
10100110000000 6.6% 92
11101010000000 3.7% 132
10011001000000 2.9% 173
10100101000000 2.8% 173
10010110000000 2.8% 173
11100110000000 1.7% 133
11011010000000 1.7% 133
11101001000000 1.7% 133
10010101000000 1.3% 254
00011010000000 1.0% 130
00100110000000 1.0% 130
11011001000000 1.0% 134
11100101000000 1.0% 134
11010110000000 1.0% 134
00101001000000 1.0% 130

In [15]:
test(40, 5, np.full(5 * 2, np.pi/4))

Iteration 10, Result: 232.8505
Iteration 20, Result: 194.956
Iteration 30, Result: 209.736
Iteration 40, Result: 169.6725
Iteration 50, Result: 164.044
